<a href="https://colab.research.google.com/github/romapavelko01/FluencyGecUCUDiploma/blob/main/gector_backtranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install requirements

It can take ~15min (long torch install)
Uncomment and run cells

In [ ]:
!git clone https://github.com/asivokon/unlp-2023-shared-task.git
!git clone https://github.com/MaksTarnavskyi/gector-large.git

Cloning into 'unlp-2023-shared-task'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 125 (delta 76), reused 98 (delta 62), pack-reused 0
Receiving objects: 100% (125/125), 4.69 MiB | 9.98 MiB/s, done.
Resolving deltas: 100% (76/76), done.
Cloning into 'gector-large'...
remote: Enumerating objects: 294, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 294 (delta 30), reused 30 (delta 30), pack-reused 254
Receiving objects: 100% (294/294), 11.34 MiB | 11.61 MiB/s, done.
Resolving deltas: 100% (164/164), done.


In [ ]:
# it is important to put the environment in the 3.8 version of Python so that there will be no errors regarding dependencies
def change_to3_8():
  !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 4
  !sudo apt install python3-pip
  !sudo apt-get install python3-distutils
  !sudo apt-get install python3-apt
  !sudo apt-get install --reinstall python3-distutils
  !sudo apt-get update
  #!pip install -r drive/MyDrive/UNLP/gector-large/requirements.txt
  !pip3 install torch==1.7.1
  !pip3 install allennlp==0.8.4
  !pip3 install python-Levenshtein==0.12.1
  !pip3 install transformers==4.2.2
  !pip3 install sentencepiece==0.1.95
  !pip3 install scikit-learn==0.22.2
  !pip3 install overrides==3.1.0
  !pip3 install numpy==1.22.4

change_to3_8()

update-alternatives: using /usr/bin/python3.8 to provide /usr/bin/python3 (python3) in auto mode
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  python-pip-whl python3-setuptools python3-wheel
Suggested packages:
  python-setuptools-doc
The following NEW packages will be installed:
  python-pip-whl python3-pip python3-setuptools python3-wheel
0 upgraded, 4 newly installed, 0 to remove and 24 not upgraded.
Need to get 2,389 kB of archives.
After this operation, 4,933 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 python-pip-whl all 20.0.2-5ubuntu1.8 [1,805 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 python3-setuptools all 45.2.0-1ubuntu0.1 [330 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 python3-wheel all 0.34.2-1ubuntu0.1 [23.9 kB]
Get:4 http://archive.ubuntu.com/ubuntu f

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 1.5 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.1-cp38-cp38-linux_x86_64.whl size=166633 sha256=c8a78d234978bce3c97c066b9c231bac14a0629a49e74585bb6d013b84026c56
  Stored in directory: /root/.cache/pip/wheels/ee/4a/07/200f37357a7f431d315287e49bdbc64c65bdadda705bfad1d9
Successfully built python-Levenshtein
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 2.0 MB/s 
     |████████████████████████████████| 880 kB 55.7 MB/s 
     |████████████████████████████████| 2.9 MB 61.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895255 sha256=1ce4fe837c48722471201d613c60a0ac8f134faa5ffb0a07681cd868ca17ae7a
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e8

In [ ]:
!python --version

Python 3.8.10


# Prepare data for training

## Merging backtranslated data and data from unlp-2023
Since gector's preprocessing requires the input file to be in the *.tok* format, we locally conduct tokenization using unlp's tokenization function, and pass the tokenized file here

In [ ]:
import os
back_tr_path = 'btr.source.tok'
target_train_path = 'unlp-2023-shared-task/data/gec-fluency/train.tgt.tok'
source_train_path = 'unlp-2023-shared-task/data/gec-fluency/train.src.tok'

tg = open(target_train_path).read()
src = open(source_train_path).read()
back_tr = open(back_tr_path).read()
len(tg.split('\n')), len(src.split('\n')), len(back_tr.split('\n'))

(32735, 32735, 32736)

In [ ]:
back_tr.split('\n')[-1]

''

In [ ]:
merged_src = src + '\n' + back_tr.rstrip('\n')
merged_tgt = tg + '\n' + tg

len(merged_src.split('\n')), len(merged_tgt.split('\n'))

(65470, 65470)

### Writing to temporary files

In [ ]:
with open('merged_train.src.tok', 'w') as m_src:
  for line in merged_src.split('\n'):
    m_src.write(line + '\n')
m_src.close()

with open('merged_train.tgt.tok', 'w') as m_tgt:
  for line in merged_tgt.split('\n'):
    m_tgt.write(line + '\n')
m_tgt.close()

In [ ]:
path = "/content/drive/MyDrive/UNLP"

import subprocess

def run_command(command_text: str):
  return subprocess.run(command_text.split())

def check_dir(path):
  os.makedirs(path, exist_ok=True)

# check_dir(path)
# check_dir(path+'/pro_data/gec-fluency/')
# check_dir(path+'/models/')
# check_dir(path+'/predictions/')

command = f'python gector-large/utils/preprocess_data.py -s merged_train.src.tok -t merged_train.tgt.tok -o {path}/pro_data/gec-fluency/merged_train'
run_command(command)

# command = f'python gector-large/utils/preprocess_data.py -s unlp-2023-shared-task/data/gec-fluency/train.src.tok -t unlp-2023-shared-task/data/gec-fluency/train.tgt.tok -o {path}/pro_data/gec-fluency/train'
# run_command(command)

# command = f'python gector-large/utils/preprocess_data.py -s unlp-2023-shared-task/data/gec-fluency/valid.src.tok -t unlp-2023-shared-task/data/gec-fluency/valid.tgt.tok -o {path}/pro_data/gec-fluency/valid'
# run_command(command)

CompletedProcess(args=['python', 'gector-large/utils/preprocess_data.py', '-s', 'merged_train.src.tok', '-t', 'merged_train.tgt.tok', '-o', '/content/drive/MyDrive/UNLP/pro_data/gec-fluency/merged_train'], returncode=0)

In [ ]:
p = '/content/drive/MyDrive/UNLP/pro_data/gec-fluency/'
tr = open(p + 'train').read()
m_tr = open(p + 'merged_train').read()
len(tr.split()), len(m_tr.split())

(3452134, 787677)